In [5]:
!pip install langchain_community langchain-openai faiss-cpu

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os

# Set your OpenAI API key
# Set your OpenAI key
from google.colab import userdata
OPENAI_API_KEY = userdata.get("OpenAI-key")

# Create a sample text file
with open("state_of_the_union.txt", "w") as f:
    f.write("""
    Hello, my fellow citizens. Today, I want to talk about the future of our great nation.
    We are facing many challenges, including climate change, economic inequality, and global competition.
    Climate change is one of the most pressing issues of our time, and we must act now to ensure
    a better future for our children and grandchildren.
    """)

# 1️⃣ Load and split documents
loader = TextLoader("state_of_the_union.txt")  # Replace with your text file
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# 2️⃣ Create embeddings and build the vector store
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
db = FAISS.from_documents(texts, embeddings)

# 3️⃣ Set up the retriever and LLM
retriever = db.as_retriever()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

# 4️⃣ Build the chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 5️⃣ Ask a question
query = "What did the president say about climate change?"
result = qa_chain(query)

# Output the result
print("Answer:", result["result"])
print("Source Documents:", result["source_documents"])

<ipython-input-4-fb26833ba234>:36: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)
<ipython-input-4-fb26833ba234>:48: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)


Answer: The president emphasized that climate change is one of the most pressing issues of our time and highlighted the importance of taking action now to secure a better future for future generations.
Source Documents: [Document(id='67d81638-d6a8-4b49-8f07-385e7b117cba', metadata={'source': 'state_of_the_union.txt'}, page_content='Hello, my fellow citizens. Today, I want to talk about the future of our great nation.\n    We are facing many challenges, including climate change, economic inequality, and global competition.\n    Climate change is one of the most pressing issues of our time, and we must act now to ensure\n    a better future for our children and grandchildren.')]
